# 获取所有专辑歌曲URL

In [1]:
import cookielib
import urllib2
import time
import random
import datetime
from bs4 import *
import re


BASE_URL = 'http://www.jitashe.net'


In [2]:
def get_soup(url):  
    try:
        response = urllib2.urlopen(baseUrl,timeout=10)  # 请求网页，返回句柄
        page = response.read()  # 读取并返回网页内容
        # print 'open !!!'    
        soup = BeautifulSoup(page)
        return soup

    except Exception, e:
        print e
        print url


In [3]:
def find_all_alblum_name(soup):
    
    album_list=[]
    albumlist = soup.findAll('a',{'class':'CDcover100'})
    for album_li in albumlist:
        album={}
        album['url']= BASE_URL + album_li['href']
        album['name'] = album_li['title']
        album_list += [album]   
        
    return album_list

baseUrl = 'http://www.jitashe.net/album/list/4447'
soup= get_soup(baseUrl)
album_list =  find_all_alblum_name(soup)
album_list

C:\Users\wybert\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[{'name': u'\u518d\u4e5f\u4e0d\u4f1a\u53bb\u4e3d\u6c5f',
  'url': 'http://www.jitashe.net/album/2100205356/'},
 {'name': u'\u5409\u59c6\u9910\u5385',
  'url': 'http://www.jitashe.net/album/2013969781/'},
 {'name': u'\u8d75\u5c0f\u96f7',
  'url': 'http://www.jitashe.net/album/465009/'}]

In [4]:
def find_song_url(url):
    
    song_list = []
    
    response = urllib2.urlopen(url,timeout=10)  # 请求网页，返回句柄
    page = response.read()  # 读取并返回网页内容
    
    regular = r'''<div class='song_info'>[\s\S]{10,300}</div>'''
    pattern = re.compile(regular)
    
    song_div_list = pattern.findall(page)
    for song in song_div_list:   
        asong={}
        soup = BeautifulSoup(song)
        song = soup.find('a')
        asong['url']=BASE_URL + song['href']
        asong['name']=song.text
        song_list +=[asong]
    return song_list

URL_LIST=[]

for item  in album_list:
    url = item['url']
#     print url
    temp={}
    song_list = find_song_url(url)
    temp['album']= item['name']
    temp['datial']=song_list
    URL_LIST+=[temp]
#     brebak
    
    

# 获取吉他谱URL

In [5]:

def get_tab_url(url):
    tab_list = []
    
    response = urllib2.urlopen(url,timeout=10)  # 请求网页，返回句柄
    page = response.read()  # 读取并返回网页内容

    regular = r'''<div class="bm_c tablist">[\s\S]*<div class="bm" id="songvideo">'''

    pattern = re.compile(regular)
    tab_div = pattern.findall(page)[0].strip('''<div class="bm" id="songvideo">''')
    tab_div = BeautifulSoup(tab_div) 
    tablist = tab_div.findAll('a',{'class':'xst'})
    tab_type = tab_div.findAll('img')
#     print tab_type
    for item,type_ in zip(tablist,tab_type):
        tab={}
        tab['url'] = BASE_URL+item['href']
        tab['type'] = type_['title']
        tab_list += [tab]        
    return tab_list

def get_img_url(url):
    img_list = []
    
    response = urllib2.urlopen(url,timeout=10)  # 请求网页，返回句柄
    page = response.read()  # 读取并返回网页内容

    regular = r'''<div class="imgtab">[\s\S]*<a name="forDownload" id="forDownload"></a>'''
    pattern = re.compile(regular)
    img_div = pattern.findall(page)[0].strip('''<a name="forDownload" id="forDownload"></a>''')
    img_div = BeautifulSoup(img_div) 
    imglist = img_div.findAll('img')
    for item in imglist:
#         print item
        img={}
        img['url'] = BASE_URL+'/'+item['file']
        img_list += [img]
        
    return img_list
'''
抓取网页文件内容，保存到内存

@url 欲抓取文件 ，path+filename
'''
def get_file(url):
    try:
        response = urllib2.urlopen(url,timeout=10)  # 请求网页，返回句柄
        data=response.read()
        return data
    except BaseException, e:
        print e
        return None
    
'''
保存文件到本地

@path  本地路径
@file_name 文件名
@data 文件内容
'''
from os import mkdir
def save_file(path, file_name, data):
    if data == None:
        return
    
    if(not path.endswith("/")):
        path=path+"/"
    file=open(path+file_name, "wb")
    file.write(data)
    file.flush()
    file.close()
    

In [6]:

def download_asong_tab(song_album,song_name,song_url):
    tab_url = get_tab_url(song_url)
    for j,item in enumerate(tab_url):
        if item['type'] !=u'图片谱':
            continue
        imgurl = item['url']
        img = get_img_url(imgurl)
        for i,jpg_url in enumerate(img):
            file_data = get_file(jpg_url['url'][:-10]) 
            file_name=song_name+u'版本'+str(j+1)+'_'+str(i+1)+'.jpg'
            save_file(song_album, file_name, file_data)


In [11]:
for item in album_list:
    mkdir(item['name'])

In [7]:
for item in URL_LIST[-1:]:
    print '获取专辑',item['album']
    print '*'*10
    song_album=item['album']
    
    for song in item['datial']:
        song_name=song['name']
        song_url=song['url']
        try:
            print song_name
            download_asong_tab(song_album,song_name,song_url)
        except Exception,e:
            print e 
#     break

获取专辑 赵小雷
**********
人家
list index out of range
未给姐姐递出的信
list index out of range
çť
不开的唇
赵小雷
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
南方姑娘
list index out of range
Over
list index out of range
开往北京的火车
list index out of range
背影
list index out of range
妈妈
南方姑娘 (弹唱版)
list index out of range
民谣
list index out of range


In [42]:
URL_LIST[0]

{'album': u'\u518d\u4e5f\u4e0d\u4f1a\u53bb\u4e3d\u6c5f',
 'datial': [{'name': u'\u518d\u4e5f\u4e0d\u4f1a\u53bb\u4e3d\u6c5f',
   'url': 'http://www.jitashe.net/song/1774824066/'}]}

In [8]:
filehander = open('test.txt','w')


In [ ]:
filehander.write('sjhd')